In [1]:
import pandas as pd
import numpy as np
import re
import time
import os
import shutil

from selenium import webdriver

from tqdm.notebook import tqdm

import json
from datetime import datetime

# Get parsed videos

In [2]:
df = pd.read_json('dados_brutos/parsed_videos_2020_10_15_20_24_29.json', lines=True)
df.head()

,link,title,query
0,https://www.youtube.com/watch?v=0CFl1ZRc5TI,Deep Learning - Lecture 1.2,machine+learning
1,https://www.youtube.com/watch?v=OCHbm88xUGU,Deep Learning - Lecture 1.1,machine+learning
2,https://www.youtube.com/watch?v=9_V7a_8-5k0,Bobby Samuel on Machine Learning and AI in Hea...,machine+learning
3,https://www.youtube.com/watch?v=M8ueP_I6jeo,🤖 O que é Machine Learning? Quais impactos da ...,machine+learning
4,https://www.youtube.com/watch?v=1ApEHKYR9Rg,Practical Machine Learning Lecture 2 - 2020,machine+learning


# Create a column to hold only the video code

In [3]:
dummy = df.copy()
dummy['code'] = df['link'].str.extract(r'(v=.*)')
dummy[['link', 'code']]

,link,code
0,https://www.youtube.com/watch?v=0CFl1ZRc5TI,v=0CFl1ZRc5TI
1,https://www.youtube.com/watch?v=OCHbm88xUGU,v=OCHbm88xUGU
2,https://www.youtube.com/watch?v=9_V7a_8-5k0,v=9_V7a_8-5k0
3,https://www.youtube.com/watch?v=M8ueP_I6jeo,v=M8ueP_I6jeo
4,https://www.youtube.com/watch?v=1ApEHKYR9Rg,v=1ApEHKYR9Rg
...,...,...
2825,https://www.youtube.com/watch?v=EKs4dvnftPA,v=EKs4dvnftPA
2826,https://www.youtube.com/watch?v=TQrJthPRkXc,v=TQrJthPRkXc
2827,https://www.youtube.com/watch?v=D0nvEqeQ5as,v=D0nvEqeQ5as
2828,https://www.youtube.com/watch?v=vvVJU_ONNd0,v=vvVJU_ONNd0


# Detecting and removing duplicates

In [78]:
dup = dummy.groupby(['code', 'query'], as_index=False).size().to_frame(name='count').reset_index()
print('Total of duplicated samples:', dup[dup['count'] > 1]['count'].sum())
print('Unique duplicated samples:', dup[dup['count'] > 1].shape[0])
print('Total of samples in data:', dummy.shape[0])
print('Total after deleting duplicated samples:', ((dummy.shape[0] - dup[dup['count'] > 1]['count'].sum()) + dup[dup['count'] > 1].shape[0]))

Total of duplicated samples: 2106
Unique duplicated samples: 119
Total of samples in data: 2830
Total after deleting duplicated samples: 843


In [79]:
df_uniques = df.drop_duplicates()
df_uniques.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 843 entries, 0 to 2357
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    842 non-null    object
 1   title   843 non-null    object
 2   query   843 non-null    object
dtypes: object(3)
memory usage: 26.3+ KB


# Remove link with null value

In [96]:
df_clean = df_uniques[df_uniques['link'].notna()]
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 842 entries, 0 to 2357
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    842 non-null    object
 1   title   842 non-null    object
 2   query   842 non-null    object
dtypes: object(3)
memory usage: 26.3+ KB


# Getting video information

In [99]:
curr_date = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

# Iterate by row in data frame
for index, row in tqdm(df_uniques.iterrows(), desc='Getting video information'):
    driver = webdriver.Firefox()

    driver.get(row['link'])

    print('Views:', driver.find_element_by_class_name('view-count'))
    
    driver.quit()
    '''for page in tqdm(range(1, 101), desc=description):
        urll = url.format(query=query, page=page)
        
        driver = webdriver.Firefox()
        
        driver.get(urll)
        
        # Get all video links
        links = driver.find_elements_by_id('video-title')
        
        for link in links:
            video_link = link.get_attribute('href')
            title = link.get_attribute('title')
                    
            with open(f'dados_brutos/parsed_videos_{curr_date}.json', 'a+') as output:
                data = {'link': video_link, 'title': title, 'query': query}
                output.write('{}\n'.format(json.dumps(data)))
        
        driver.quit()'''

NoSuchElementException: Message: Unable to locate element: .view-count
